In [ ]:
pip install ultralytics opencv-python torch torchvision

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Load YOLOv8 model for vehicle detection
yolo_model = YOLO('yolov8n.pt')  # You can replace with the path to your specific YOLOv8 model

# Define a simple CNN for mirror detection (placeholder)
def build_mirror_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Binary classification: Mirror or not
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Load the CNN model for mirror detection (train separately with mirror/non-mirror images)
mirror_cnn = build_mirror_cnn()

# Initialize video capture (0 is usually the default webcam)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Step 1: Vehicle detection using YOLOv8
    results = yolo_model(frame)

    # Extract detected vehicle bounding boxes from YOLOv8 output
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            class_id = box.cls[0]

            # Assuming class_id corresponds to vehicle classes (cars, trucks, etc.)
            if class_id in [2, 3, 5, 7]:  # COCO classes for cars, trucks, buses, etc.
                # Draw bounding box around the detected vehicle
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"Vehicle {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                # Step 2: Mirror detection using CNN (on the vehicle region)
                vehicle_region = frame[y1:y2, x1:x2]  # Crop vehicle region
                vehicle_region_resized = cv2.resize(vehicle_region, (64, 64))  # Resize to fit CNN input size
                vehicle_region_resized = np.expand_dims(vehicle_region_resized, axis=0)  # Add batch dimension

                # Predict if mirrors are present in the vehicle region using the CNN
                mirror_prediction = mirror_cnn.predict(vehicle_region_resized)
                if mirror_prediction > 0.5:
                    cv2.putText(frame, "Mirror Detected", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                else:
                    cv2.putText(frame, "No Mirror", (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Vehicle and Mirror Detection', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()